In [4]:
stores = [
    {
        'name': 'My_first_store',
        'items': [
            {
                'name': 'sample_item',
                'price': 10.0
            }
        ]
    }
]

In [12]:
from flask import Flask, jsonify, request

app = Flask(__name__)


# POST /store data: {name}
@app.route('/store',methods=['POST'])
def create_store():
    new_data = request.get_json()
    new_store = {
        'name': new_data['name'],
        'items': []
    }
    stores.append(new_store)
    return jsonify(new_store)

# GET /store/<string:name>
@app.route('/store/<string:name>',methods=['GET'])
def return_store(name):
    for store in stores:
        if name == store['name']:
            return jsonify(store)
    return jsonify({'message': 'Sorry! Store not found!'})

# GET /store
@app.route('/store',methods=['GET'])
def return_stores():
    return jsonify({'stores': stores})

# POST /store/<string:name>/item {name:,price:}
@app.route('/store/<string:name>/item',methods=['POST'])
def create_item_in_store(name):
    new_data = request.get_json()
    for store in stores:
        if name == store['name']:
            new_item = {
                'name': new_data['name'],
                'price': new_data['price']
            }
            store['items'].append(new_item)
            return jsonify(store)
    return jsonify({'message': "Error!!! Store not found"})

# GET /store/<string:name>/item
@app.route('/store/<string:name>/item',methods=['GET'])
def return_item_in_store(name):
    for store in stores:
        if name == store['name']:
            if len(store['items']) == 0:
                return jsonify({'message': "Sorry! There are no items in this store. Come back again later."})
            return jsonify({"Items in store": store['items']})
    return jsonify({'message': 'Sorry! Store not found!'})

@app.route('/')
def home():
    return "hello world!\nWelcome to the mall."

app.run(port=5555)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5555/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2020 16:43:12] "POST /store/Cheesstore/item HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 16:43:20] "POST /store/Cheese%20store/item HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 16:43:29] "GET /store HTTP/1.1" 200 -


In [1]:
class User:
    def __init__(self, _id, username, password):
        self.id = _id
        self.username = username
        self.password = password
    

users = [
    User(1,"siva","abcd")
]

username_mapping = {u.username: u for u in users}

userid_mapping = {u.id: u for u in users}

def authenticate(username, password):
    user = username_mapping.get(username, None)
    if user and user.password == password:
        return user

def identity(payload):
    user_id = payload['identity']
    return userid_mapping.get(user_id, None)

In [11]:
### RESTful api

from flask import Flask,jsonify,request
from flask_restful import Resource, Api, reqparse
from flask_jwt import JWT, jwt_required

app = Flask(__name__)
api = Api(app)
app.secret_key = "bottle"

jwt = JWT(app, authenticate, identity)

Items = []


class Item(Resource):
    parser = reqparse.RequestParser()
    parser.add_argument("price",required=True,type=float,help="This filed cant be left empty")
        
    @jwt_required()
    def get(self, name):
        item = next(filter(lambda x: x['name'] == name, Items), None)
        return {'Item': item}, 200 if item else 404
    
    def post(self,name):
        if next(filter(lambda x: x['name'] == name, Items), None) is not None:
            return {"message": "An item with name {} already exists.".format(name)}, 400

        data = Item.parser.parse_args()
        new_item = {
            "name": name,
            "price": data['price']
        }
        Items.append(new_item)
        return new_item, 201
    
    def delete(self,name):
        global Items
        Items = list(filter(lambda x: x['name'] != name, Items))
        return {"message": "Item deleted!"}
    
    def put(self,name):
        data = Item.parser.parse_args()
        
        item = next(filter(lambda x: x['name'] == name, Items), None)
        if item is not None:
            item.update(data)
        else:
            item = {'name': name, 'price': data['price']}
            Items.append(item)
        return item
    
class ItemList(Resource):
    def get(self):
        return {"Items": Items}
    
api.add_resource(Item, '/items/<string:name>')
api.add_resource(ItemList,'/items')

app.run(port=5555)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5555/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Apr/2020 02:56:19] "PUT /items/crm HTTP/1.1" 400 -
127.0.0.1 - - [15/Apr/2020 02:56:32] "PUT /items/crm HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 02:56:38] "GET /items HTTP/1.1" 200 -
